In [2]:
import torch
import torch.nn as nn
import torch.nn.functional
import torch_geometric.nn 
from torch_geometric.utils import from_networkx
from torch_geometric.data import Data
import pandas as pd
import networkx as nx
import numpy as np
import tqdm
import random
from torch_geometric.nn import GATv2Conv, global_mean_pool
from torch.utils.data import Dataset
from torch_geometric.loader import DataLoader


In [3]:
pkl_graphs = pd.read_pickle('/Users/MathildeStouby/Desktop/P5 GitHub/5-semester/Momentum graphs.pkl')

In [4]:
unique_nodes = []
for value in pkl_graphs.values():
    graph = value['graph']

    temp = [node for node in graph.nodes() if node not in unique_nodes]
    unique_nodes.extend(temp)

In [5]:
idx_to_pos = dict(enumerate(unique_nodes))
pos_to_idx = {pos : idx for idx, pos in idx_to_pos.items()}

In [6]:
pyg_data = []

#add node attributes
for value in pkl_graphs.values():
    momentum = value['momentum']

    graph = value['graph']

    closeness = nx.closeness_centrality(graph)
    betweenness = nx.closeness_centrality(graph)
    pagerank = nx.pagerank(graph, weight='weight')
    centrality_list = [closeness, betweenness, pagerank] 

    adj_dict = nx.to_dict_of_dicts(graph)

    for node in list(graph.nodes()):
        adj_vect = np.zeros((len(unique_nodes)))
        players = adj_dict[node]
        for key, value in players.items():
            adj_vect[pos_to_idx[key]] = value['weight']
        adj_vect = torch.from_numpy(adj_vect).float()
        centrality_vect = []
        for measure in centrality_list:
            if node in list(measure.keys()):
                centrality_vect.append(measure[node])
            else:
                centrality_vect.append(0)
        centrality_vect = torch.Tensor(centrality_vect).float()        
        #graph.nodes[node]['x'] = torch.cat((adj_vect, centrality_vect), -1)
        graph.nodes[node]['x'] = centrality_vect

   
    for node in unique_nodes:
        if node not in graph.nodes:
            graph.add_node(node) 
            #graph.nodes[node]['x'] = torch.from_numpy(np.zeros((len(unique_nodes)+3))).float()
            graph.nodes[node]['x'] = torch.from_numpy(np.zeros(3)).float()  
            

    data = from_networkx(graph)
    data['y'] = momentum
    try:
        data['weight']
        pyg_data.append(data)
    except:
        print(data)
   

Data(x=[23, 3], edge_index=[2, 0], y=-0.0141746799999999)
Data(x=[23, 3], edge_index=[2, 0], y=0.00802374)


In [7]:
train_idx = random.sample(range(len(pyg_data)), int(len(pyg_data) * 0.8))
test_idx = [i for i in range(len(pyg_data)) if i not in train_idx]

In [8]:
train_data = [pyg_data[idx] for idx in train_idx]
dataloader = DataLoader(train_data, batch_size=64, shuffle=True)

In [9]:
class GAT(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_heads, v2 = True):
        super(GAT, self).__init__()
        self.layer1= GATv2Conv(input_dim, hidden_dim, heads=num_heads)
        self.layer2= GATv2Conv(hidden_dim * num_heads, hidden_dim, heads=num_heads)
        self.layer3= GATv2Conv(hidden_dim * num_heads, hidden_dim, heads=num_heads)
        self.layer4= GATv2Conv(hidden_dim * num_heads, hidden_dim, heads=num_heads)
        self.layer5 = GATv2Conv(hidden_dim * num_heads, output_dim, heads=1, concat=False)
        self.activation_function = nn.ReLU()
     

    def forward(self, input, edge_index, batch):
        output = self.activation_function(self.layer1(input, edge_index))
        output = self.activation_function(self.layer2(output, edge_index))
        output = self.activation_function(self.layer3(output, edge_index))
        output = self.activation_function(self.layer4(output, edge_index))
        output = self.layer5(output, edge_index)
        output = global_mean_pool(output, batch)
        return output

In [10]:
#input_dim = len(unique_nodes)+3
input_dim = 3
lr = 0.01

gat = GAT(input_dim = input_dim, hidden_dim = 11, output_dim = 1, num_heads = 6)
optimizer = torch.optim.SGD(gat.parameters(), lr=lr)
loss_fn = torch.nn.MSELoss()
epochs_num = 100

In [11]:
for epoch in tqdm.tqdm(range(epochs_num)):
    
    epoch_loss = 0
    for batch in tqdm.tqdm(dataloader):
        # Forward pass
        optimizer.zero_grad()
        
        output = gat(batch.x, batch.edge_index, batch.batch)
        
        # Calculate loss
        loss = loss_fn(output, batch.y)
        loss.backward()

        optimizer.step()
        
        epoch_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {epoch_loss/len(dataloader):.4f}") 
gat.eval() 

  0%|          | 0/100 [00:00<?, ?it/s]/Users/MathildeStouby/opt/anaconda3/lib/python3.9/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/MathildeStouby/opt/anaconda3/lib/python3.9/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([22])) that is different to the input size (torch.Size([22, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
  1%|          | 1/100 [00:01<02:02,  1.23s/it]

Epoch 1, Loss: 0.0002


  2%|▏         | 2/100 [00:02<01:34,  1.04it/s]

Epoch 2, Loss: 0.0001


  3%|▎         | 3/100 [00:02<01:28,  1.09it/s]

...

 99%|█████████▉| 99/100 [01:20<00:00,  1.22it/s]

Epoch 99, Loss: 0.0001


100%|██████████| 100/100 [01:21<00:00,  1.22it/s]

Epoch 100, Loss: 0.0001


GAT(
  (layer1): GATv2Conv(3, 11, heads=6)
  (layer2): GATv2Conv(66, 11, heads=6)
  (layer3): GATv2Conv(66, 11, heads=6)
  (layer4): GATv2Conv(66, 11, heads=6)
  (layer5): GATv2Conv(66, 1, heads=1)
  (activation_function): ReLU()
)

In [12]:
from sklearn.metrics import mean_absolute_percentage_error

y_pred  = []
y_true = []
batch = torch.from_numpy(np.zeros(23)).long()

with torch.no_grad():
    for idx in test_idx:
        batch = torch.zeros(pyg_data[idx].x.size(0), dtype=torch.long)
        output = gat(pyg_data[idx].x, pyg_data[idx].edge_index, batch)
        y_pred.append(output.item())  # Convert to numpy for MAPE
        y_true.append(pyg_data[idx].y)

mean_absolute_percentage_error(y_true, y_pred)

1.0931291884958338